![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import matplotlib.pyplot as plt
import missingno as msno
from textblob import TextBlob
import seaborn as sns 
%matplotlib inline
sns.set()

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

Загрузим датасеты. Добавим дополнительный датасет по городам. Далее попробуем достать из него полезные нам признаки.

In [ ]:
df_train = pd.read_csv('/kaggle/input/sf-dst-restaurant-rating/main_task.csv')
df_test = pd.read_csv('/kaggle/input/sf-dst-restaurant-rating/kaggle_task.csv')
sample_submission = pd.read_csv('/kaggle/input/sf-dst-restaurant-rating/sample_submission.csv')
df_city = pd.read_csv('/kaggle/input/world-cities/worldcities.csv')
cost = pd.read_csv('/kaggle/input/2020-cost-of-living/cost of living 2020.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем, где у нас трейн
df_test['sample'] = 0 # помечаем, где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, поэтому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## Обработка признаков

Пройдемся по каждому признаку и приведем предварительную обработку. Посмотрим на пропуски и создадим новые признаки.

In [ ]:
data.isna().sum()

In [ ]:
msno.bar(data, figsize=(18, 6),  fontsize=12);

Cuisine Style, Price Range и Number of Reviews содержат много пропусков. Далее мы их заполним и создадим новые признаки, которые будут говорить об отсутствии информации.

### Restaurant_id

In [ ]:
data.Restaurant_id.value_counts()

Мы видим что есть повторящиеся id, возможно эти рестораны сетевые. 

Создадим новый признак chain: 1 - сетевой, 0 - несетевой.

In [ ]:
chain_lst = list(data.Restaurant_id.value_counts()[data.Restaurant_id.value_counts() > 1].index)

In [ ]:
data['chain'] = data[data.Restaurant_id.isin(chain_lst)].Restaurant_id.apply(lambda x: 1)
data['chain'].fillna(0, inplace=True)

In [ ]:
data['chain'].value_counts()

### City

In [ ]:
plt.figure(figsize=(10,4))
data.City.value_counts(ascending=False).plot(kind='bar');

Создадим список городов и найдем в добавленом датасете эти города.

In [ ]:
data.loc[data.City == 'Oporto', 'City'] = 'Porto'
city_lst = data.City.unique()

In [ ]:
data['ranked_cities'] = data['City'].rank()

In [ ]:
df_city = df_city[(df_city.city_ascii.isin(city_lst)) &
                  (df_city.country != 'United States') &
                  (df_city.country != 'Canada') &
                  (df_city.country != 'Venezuela')]

In [ ]:
set(city_lst) - set(df_city.city_ascii) # проверяем все ли города нашли

Удалим ненужные колонки, заменим пропуски и переведем численость в млн. человек. Признак столица заменим числовыми значениями на 0 и 1

In [ ]:
df_city = df_city.drop(['city', 'iso2', 'iso3', 'admin_name',  'id'], axis=1)

In [ ]:
df_city['population']  = round((df_city['population'] / 1000000), 2)

In [ ]:
df_city.capital.fillna('primary', inplace=True)

In [ ]:
df_city['capital'] = df_city.capital.apply(lambda x: 1 if x == 'primary' else 0)

Посмотрим на датасет уровня жизни

In [ ]:
country_lst = df_city['country'].tolist() 

In [ ]:
cost.head(3)

In [ ]:
cost.loc[cost.Country.str.contains('Czec'), 'Country'] = 'Czechia'
cost = cost[cost.Country.isin(country_lst)]

In [ ]:
set(country_lst) - set(cost.Country)

In [ ]:
cost.drop('Unnamed: 9', axis=1, inplace=True)
plt.figure(figsize=(8,7))
sns.heatmap(cost.corr(), annot=True, cmap='vlag');

Все признаки хорошо скоррелированны между собой. Выбрал несколько признаков, которые на мой взгляд могут улучшить модель.

In [ ]:
cost.columns

In [ ]:
cost.drop(['Rent Index', 'Cost of Living Plus Rent Index',
           'Groceries Index', 'Local Purchasing Power Index' ], axis=1, inplace=True)

Объединим датасеты в один

In [ ]:
df_city = df_city.join(cost.set_index('Country'), on='country')

In [ ]:
data = data.join(df_city.set_index('city_ascii'), on='City')

Создадим признак количество ресторанов в городе.

In [ ]:
city_counts_dict = dict(data['City'].value_counts())

In [ ]:
data['rest_counts'] = data.City.map(city_counts_dict)

In [ ]:
plt.figure(figsize=(18, 7))
sns.boxplot(x='City', y='Rating', data=data[data['sample'] == 1])
plt.xticks(rotation=90);

На графике можно увидеть, что рейтинг ресторанов в Rome начинаются с 3, а в Milan вероятность рейтинга с оценкой 5 очень мала. 

### Cuisine Style

In [ ]:
data['Cuisine Style'][1]

Значения представлены в виде строки, обработаем их. Создадим список уникальных кухонь представленных во всем датасете. Пропуски заполним значением 'Other', создадим колонку, которая показывает было ли значение пустое.

In [ ]:
data['Cuisine Style_NAN'] = data['Cuisine Style'].isna().astype('uint8')

In [ ]:
data['Cuisine Style'].fillna("['Other']", inplace=True)

In [ ]:
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: eval(x))

Создадим признак 'cuisine_count' - количество представленных кухонь в ресторане.

In [ ]:
data['cuisine_count'] = data['Cuisine Style'].apply(lambda x: len(x))

Посмотрим частоту встречаемости кухонь и выберем редкие (< 50)

In [ ]:
plt.figure(figsize=(18,6))
data.explode('Cuisine Style')['Cuisine Style'].value_counts(ascending=False).plot(kind='bar');

vegetarian friendly и european лидируют. Кухни, встречающиеся меньше 50 раз, будем считать редкими.

In [ ]:
cuisine_rare_lst = data.explode('Cuisine Style')['Cuisine Style'].value_counts()[
    data.explode('Cuisine Style')['Cuisine Style'].value_counts() < 50].index.tolist()

In [ ]:
def cuisine_rare_count(cell):
    '''Количество редких кухонь в ресторане'''
    x = 0
    for i in cuisine_rare_lst:
        if i in cell:
            x += 1
    return x

In [ ]:
data['cuisine_rare'] = data['Cuisine Style'].apply(cuisine_rare_count)

Создадим dummy переменные кухонь

In [ ]:
cuisine_lst = list(data.explode('Cuisine Style')['Cuisine Style'].unique())

In [ ]:
def find_item(cell):
    if item in cell:
        return 1
    return 0
    
for item in cuisine_lst:
    data[item] = data['Cuisine Style'].apply(find_item)

### Ranking

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
data['Ranking'].hist(bins=100);

In [ ]:
# посмотрим на топ 10 городов
for x in (data['City'].value_counts())[0:10].index:
    data['Ranking'][data['City'] == x].hist(bins=35)

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за этого мы имеем смещение. Разделим Ranking на количество ресторанов в городе.

In [ ]:
data['ranking_norm'] = data.Ranking / data.rest_counts

In [ ]:
for x in (data['City'].value_counts())[0:10].index:
    data['ranking_norm'][data['City'] == x].hist(bins=35)

### Price Range

Заменим пропуски и изменим значения от 1 до 3. Создадим признак отсутствия информации.

In [ ]:
data['Price Range'].value_counts(dropna=False)

In [ ]:
data['Price Range_NAN'] = data['Price Range'].isna().astype('uint8')

In [ ]:
for city in city_lst:
    print(city,  data[data['City'] == city]['Price Range'].mode()[0])

Мода цен в ресторанах по городам одинаковая.

In [ ]:
data['Price Range'].fillna(data['Price Range'].mode()[0], inplace=True)

In [ ]:
price_dict = {'$': 1,
              '$$ - $$$': 2,
              '$$$$': 3}

In [ ]:
data['Price Range'] = data['Price Range'].map(price_dict)

In [ ]:
sns.boxplot(x='Price Range', y='Rating', data=data);

Самые дорогие рестораны реже получают низкие оценки.

### Reviews

In [ ]:
data.Reviews[1]

Значения признака представлены в виде строки, но есть четкая структура списков.

In [ ]:
data['Reviews'] = data.Reviews.dropna().apply(
    lambda x: eval(x.replace('nan', '').replace('[,', "['',")))

In [ ]:
data['Reviews'] = data['Reviews'].dropna().apply(
    lambda x: np.nan if len(x[0]) == 0 & len(x[1]) == 0 else x)

Создадим новые признаки, reviews и date_1 date_2



In [ ]:
data['date'] = data['Reviews'].dropna().apply(lambda x: x[1])
data['date_1'] = data['date'].dropna().apply(lambda x: x[1] if len(x) == 2 else np.nan)
data['date_2'] = data['date'].dropna().apply(lambda x: x[0] if len(x) > 0 else np.nan)

In [ ]:
data.drop('date', axis=1, inplace=True)

Заменим пропуски в date_1 и date_2 на min и max дату соответственно. Максимальная разница количества дней  между коментариями будет говорит о том, что отзывы практически не оставляют

In [ ]:
data['date_1'] = pd.to_datetime(data['date_1'])
data['date_2'] = pd.to_datetime(data['date_2'])

In [ ]:
data['date_1'].fillna(data['date_1'].min(), inplace=True)
data['date_2'].fillna(data['date_2'].max(), inplace=True)

Создадим признак разницы дней между последними двумя отзывами.

In [ ]:
data['rewiew_days'] = abs(data['date_2'] - data['date_1']).dt.days

In [ ]:
plt.figure(figsize=(10,5))
data['rewiew_days'].hist(bins=75);

In [ ]:
data['days_to_today'] = (datetime.now() - data['date_2']).dt.days
data['days_to_today']

Обработаем отзывы. Создадим признак количества последних коментариев (0-2)

In [ ]:
data['Reviews'] = data['Reviews'].dropna().apply(lambda x: x[0])

In [ ]:
data['Reviews_1'] = data['Reviews'].dropna().apply(lambda x: x[1] if len(x) == 2 else np.nan)
data['Reviews_2'] = data['Reviews'].dropna().apply(lambda x: x[0] if len(x) > 0 else np.nan)

In [ ]:
data['Reviews_2'] = data['Reviews_2'].apply(lambda x: np.nan if x == '' else x)

In [ ]:
def analysisPol(text):
    pol = TextBlob(text).polarity
    return pol

def analysisSub(text):
    sub = TextBlob(text).subjectivity
    return sub

In [ ]:
data['Reviews_1_pol'] = data['Reviews_1'].dropna().apply(analysisPol)
data['Reviews_2_pol'] = data['Reviews_2'].dropna().apply(analysisPol)

In [ ]:
data['Reviews_1_pol'].fillna(0, inplace=True)
data['Reviews_2_pol'].fillna(0, inplace=True)

In [ ]:
data['Reviews_pol_sum'] = data[['Reviews_1_pol', 'Reviews_2_pol']].sum(axis=1)

In [ ]:
data['Reviews_1_sub'] = data['Reviews_1'].dropna().apply(analysisSub)
data['Reviews_2_sub'] = data['Reviews_2'].dropna().apply(analysisSub)

In [ ]:
data['Reviews_1_sub'].fillna(0, inplace=True)
data['Reviews_2_sub'].fillna(0, inplace=True)

In [ ]:
data['Reviews_sub_sum'] = data[['Reviews_1_sub', 'Reviews_2_sub']].sum(axis=1)

In [ ]:
data['last_rewiew_counts'] = data['Reviews'].dropna().apply(
    lambda x: 2 if len(x) == 2 else 1 if len(x) == 1 else 0)

In [ ]:
data.last_rewiew_counts.fillna(0, inplace=True)

In [ ]:
def len_rewiews(cell):
    x = 0
    for i in cell:
        x += len(i)
    return x

In [ ]:
data['len_rewiews'] = data['Reviews'].dropna().apply(len_rewiews)
data['len_rewiews'].fillna(0, inplace=True)

### Number of Reviews

Посмотрим пропуски в колонке Number of Reviews

In [ ]:
data[data['Number of Reviews'].isna()][['Number of Reviews', 'last_rewiew_counts']].head()

In [ ]:
data['Number of Reviews'].isna().sum()

In [ ]:
data.loc[(data['last_rewiew_counts'] == 0) & (
    data['Number of Reviews'].isna()), 'Number of Reviews'] = 0
data.loc[(data['last_rewiew_counts'] == 1) & (
    data['Number of Reviews'].isna()), 'Number of Reviews'] = 1

In [ ]:
data['Number of Reviews'].isna().sum()

In [ ]:
data[data['last_rewiew_counts'] ==0][['Number of Reviews', 'last_rewiew_counts']].head()

У нас есть значения последних отзывов равные 0, хотя общее количество отзывов > 0. Это значит, что отзывы оставляли пустые.

Создадим словарь количество отзывов по городам и подставим в датасет



In [ ]:
data.groupby('City')['Number of Reviews'].sum().sort_values(ascending=False).plot(kind='bar');

In [ ]:
rewiews_city_dict = dict(data.groupby('City')['Number of Reviews'].sum())

In [ ]:
data['num_rewiews_city'] = data.City.map(rewiews_city_dict)

Создадим признак отношения рейтинга в городе к количеству отзывов в городе.

In [ ]:
data['relative_rank_reviews'] = data['Ranking'] / data['num_rewiews_city']

### URL_TA 

In [ ]:
data['URL_TA'][0]

Ссылки на рестораны парсить пока не умею :( удалим 

In [ ]:
data.drop(['URL_TA'], axis=1, inplace=True)

### ID_TA

In [ ]:
data['ID_TA'] = data['ID_TA'].apply(lambda x: int(x[1:]))

### dummy

In [ ]:
data = pd.get_dummies(data, columns=['City'])

### Удалим все нечисловые признаки и признаки, которые ухудшали модель.

In [ ]:
data = data.select_dtypes(exclude='O').drop(['date_1', 'date_2', 
                                             'Reviews_1_pol',
                                             'capital', 'Ranking', 'Reviews_1_sub', 'Reviews_2_sub',
                                             'chain', 'McMeal($)', 
                                             'Restaurant Price Index',
                                             'last_rewiew_counts'], axis=1)

In [ ]:
data.head()

### Посмотрим распределение целевой переменной

In [ ]:
plt.figure(figsize=(10, 5))
df_train['Rating'].value_counts(ascending=True).plot(kind='barh');

### Посмотрим распределение целевой переменной относительно признака ranking_norm

In [ ]:
plt.figure(figsize=(10, 5))
data['ranking_norm'][data['Rating'] == 5].hist(bins=100);

In [ ]:
plt.figure(figsize=(10, 5))
data['ranking_norm'][data['Rating'] < 4].hist(bins=100);

In [ ]:
for x in data.Rating.unique():
    data['ranking_norm'][data['Rating'] == x].hist(bins=35)

## Корреляция признаков
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,11)
sns.heatmap(data.drop(cuisine_lst, axis=1).corr(), cmap='vlag'); # dummy кухонь не включал

Корреляция с целевой переменной

In [ ]:
data.drop(cuisine_lst, axis=1).corr().Rating.sort_values(ascending=False).to_frame()# dummy кухонь не включал


In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем, как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

# # Округлим предсказанные значения до степени округления целевой переменной
# y_pred = np.round(y_pred*2)/2

def rating(prediction):
        if prediction < 0.25:
            return 0
        elif 0.25 < prediction <= 0.75:
            return 0.5
        elif 0.75 < prediction <= 1.25:
            return 1
        elif 1.25 <prediction <= 1.75:
            return 1.5
        elif 1.75 < prediction <= 2.25:
            return 2
        elif 2.25 < prediction <= 2.75:
            return 2.5
        elif 2.75 < prediction <= 3.25:
            return 3
        elif 3.25 < prediction <= 3.75:
            return 3.5
        elif 3.75 < prediction <= 4.25:
            return 4
        elif 4.25 < prediction <= 4.75:
            return 4.5
        else:
            return 5
        
for i in range(y_pred.size):
        y_pred[i]=rating(y_pred[i])

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(30).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
for i in range(predict_submission.size):
        predict_submission[i]=rating(predict_submission[i])

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)